In [ ]:
'''

Evaluateur de performance du modèle statistique.
Ce fichier permet à l'aide de vérification humaine de valider si si une note 
relevée comme positive est un faux positif ou non.
Cette vérification est cependant biaisé par l'interprétation du modèle de 
vérification et permet uniquement de détecter les faux positifs. Un autre modèle 
d'évaluation de performance plus perforfant à été refais pour remplacer celui-ci 

'''

In [1]:
import pandas as pd
import os

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

from IPython.display import clear_output
from spacy import displacy
import edsnlp, edsnlp.pipes as eds

input_dir = "../data_clean/"
visuals_dir = "../visuals/" 

c:\Users\matti\Documents\Centrale Supelec\EI APHP\edstuto_2025\.venv\Lib\site-packages\edsnlp\core\pipeline.py:33: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
df_person = pd.read_pickle(os.path.join(input_dir, 'df_person.pkl'))
df_bio = pd.read_pickle(os.path.join(input_dir, 'df_bio.pkl'))
df_note = pd.read_pickle(os.path.join(input_dir, 'df_note.pkl'))
df_visit = pd.read_pickle(os.path.join(input_dir, 'df_visit.pkl'))
df_condition = pd.read_pickle(os.path.join(input_dir, 'df_condition.pkl'))

In [ ]:
nlp = edsnlp.blank("eds")
nlp.add_pipe(eds.sentences())
nlp.add_pipe(eds.normalizer())
nlp.add_pipe(eds.tobacco())
nlp.add_pipe(eds.negation())
nlp.add_pipe(eds.family())

false_positive = 0
for i in range(len(df_person)):
    note = df_note.iloc[i]
    text = note['note_text']
    doc = nlp(text)

    filtered_ents_for_display = [ent for ent in doc.ents if not ent._.family]
    filtered_ents_for_display = [ent for ent in filtered_ents_for_display if not ent._.negation]

    visit_id_from_sampled_note = note['visit_occurrence_id']
    matching_visit_rows = df_visit[df_visit['visit_occurrence_id'] == visit_id_from_sampled_note]

    current_note_id_scalar = note['note_id']
    person_id = matching_visit_rows['person_id'].iloc[0]

    # Compteur pour limiter le nombre de notes positives traitées
    compteur = 0
    if filtered_ents_for_display: 
        compteur += 1
        # Affichage de la note pour l'utilisateur humain
        clear_output(wait=True)
        print(f"Note {i+1}/{len(df_person)} détectée comme positive :")
        options = {"ents": [ent.label_ for ent in filtered_ents_for_display], "colors": {"ALCOHOL": "#ffffff"}}
        displacy.render(doc, style="ent", jupyter=True, options=options)
        # Input par l'utilisateur et traitement de l'input
        user_input = input("Est-ce un faux positif ? (y/n) : ").strip().lower()
        if user_input == "y":
            false_positive += 1
        else:
            df_person.loc[df_person['person_id'] == person_id, 'fumeur'] = True
        if compteur == 20:
            print("20 notes positives traitées, arrêt du traitement.")
            break
    else:
        df_person.loc[df_person['person_id'] == person_id, 'fumeur'] = False

print(f"Total total faux positifs: {false_positive}")

Note 3/959 détectée comme positive :
